# Key Detection Result Visualizer

In [2]:
%pip install yfiles_jupyter_graphs neo4j --quiet
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.6 MB/s eta 0:00:00


In [41]:
from neo4j import GraphDatabase
from typing import Dict
from yfiles_jupyter_graphs import GraphWidget
from ipywidgets import widgets, Output

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np


uri      = "neo4j+s://0bbf8364.databases.neo4j.io"
user     = "reader"
password = "copenhagen"

driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='neo4j')
session = driver.session(database='neo4j')


class GraphWidgetWrapper:
    def __init__(self, index, query, run_custom_logic):
        self.index = index
        self.out = widgets.Output()
        self.query = query
        display(self.out)
        self.fetch_and_render(run_custom_logic)



    def fetch_and_render(self, run_custom_logic):
        result = session.run(self.query, parameters={})
        widget = GraphWidget(graph=result.graph())
        run_custom_logic(widget)
        widget.organic_layout()
        widget.set_sidebar(enabled=False)
        self.out.clear_output()
        with self.out:
            widget.show()

In [45]:

def custom_node_label_mapping(index, node):
    properties = node.get('properties', {})
    return properties.get('id', properties.get('name', ''));

def custom_relationship_label_mapping(index, node):
    properties = node.get('properties', {})
    return properties.get('label', ['']);


def run_custom_logic(widget):
    widget.node_label_mapping = custom_node_label_mapping
    widget.edge_label_mapping = custom_relationship_label_mapping


graph = GraphWidgetWrapper(1, "MATCH (n)-[e]->(m) RETURN * LIMIT 20", run_custom_logic)

Output()

In [52]:


def custom_styles_mapping_image(index: int, item: Dict):
    labelToIcon = {
        'Order': 'https://em-content.zobj.net/source/apple/391/basket_1f9fa.png',
        'DistributionPoint': 'https://em-content.zobj.net/source/apple/391/department-store_1f3ec.png',
        'Product': 'https://em-content.zobj.net/source/apple/391/package_1f4e6.png',
        'Factory': 'https://em-content.zobj.net/source/apple/391/factory_1f3ed.png',
        'Headquarter': 'https://em-content.zobj.net/source/apple/391/office-building_1f3e2.png',
        'Customer': 'https://em-content.zobj.net/source/apple/391/hospital_1f3e5.png',
        'Event': 'https://em-content.zobj.net/source/apple/391/radio-button_1f518.png'
      }
    icon = labelToIcon[item['properties']['label']]

    return {
        'image': icon
    }

def run_custom_logic(widget):
    widget.node_label_mapping = custom_node_label_mapping
    widget.edge_label_mapping = custom_relationship_label_mapping
    widget.set_node_styles_mapping(custom_styles_mapping_image)

graph = GraphWidgetWrapper(2, "MATCH (n)-[e]->(m) RETURN * LIMIT 20", run_custom_logic)


Output()

In [54]:
graph = GraphWidgetWrapper(1, "MATCH (order:Order)-[h:HAS_EVENT]-(n)-[e:NEXT]->(m) RETURN *", run_custom_logic)

Output()

In [57]:
def custom_factor_mapping(node: Dict):
    return max(2,node['properties']['totalEvents'])

def run_custom_logic(widget):
    widget.node_label_mapping = custom_node_label_mapping
    widget.edge_label_mapping = custom_relationship_label_mapping
    widget.set_node_styles_mapping(custom_styles_mapping_image)
    widget.set_node_scale_factor_mapping(custom_factor_mapping)


graph = GraphWidgetWrapper(1, "MATCH (d:DistributionPoint)-[e:DELIVERS_TO]->(d2:DistributionPoint) RETURN *", run_custom_logic)

Output()